In [212]:
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os.path
try:
    from urllib2 import urlopen
except ImportError:
    from urllib.request import urlopen

### Helper Functions ###

#### Step 0: Load the helper functions below, you will be using them later to evaluate your logic. 
It is very IMPORTANT that you DONOT change these functions or your final submission will not evaluate correctly. The functions to modify are mentioned below

* Dataset File Download Function

In [213]:
#### LOAD THE HELPER FUNCTIONS BELOW####
#### IMPORTANT: DONOT change these functions or your final submission will not evaluate correctly###

## This downloads your datafile, Do not change this function
def downloadFile(dataSetId):
      fileName = '%s.csv' % (dataSetId)
      url = 'https://raw.githubusercontent.com/Auquan/qq-winter2019/master/' + fileName

      response = urlopen(url)
      status = response.getcode()
      if status == 200:
          print('Downloading the dataset %s' % (fileName))
          with open(fileName, 'w') as f:
              f.write(response.read().decode('utf8'))
          return True
      else:
          logError('File not found. Please ensure you are working with correct data set Id')
          return False

    

* Reward Function

In [214]:
## This calculates the reward, Do not change this function
def getReward(wt, wt_1, ri, l, k):
    if ri is None:
        port_returns.append(0)
        sharpe_ratio.append(0)
        port_volatility.append(0)
        reward = 0
    else:
        ri.fillna(0, inplace=True)
        returns = np.dot( ri, wt_1)
        port_returns.append(returns)
        downside_returns = [x for x in port_returns if x < 0]
        volatility = np.std(downside_returns) #np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights)))
        sharpe = np.sum(port_returns) / volatility if volatility!=0 else 0
        sharpe_ratio.append(sharpe)
        port_volatility.append(volatility)
        phi = k*(wt - wt_1).abs().sum()
        reward = returns - l*volatility - phi
        print(returns, volatility, phi, sharpe, reward)
    return reward
            

* Check Constraints Functions

In [215]:
## Do not change this function, this verifies if all constraints are met
def checkConstraints(wt, wt_1, wi, Dt, St, Qt, g, U, t, T, P, delta, chi, eta):
    violated = False
    if int(wt.sum())!=1:
        print("Fully Invested Constraint Violated: Sum of weights is not 1")
        violated = True
    if (wt>np.maximum(g, 1/float(len(wi)))).any():
        print("Diversification Constraint Violated: All weights are not less than parameter %.2f"%np.maximum(g, 1/float(len(wi))))
        violated = True
    if wt_1 is None:
        turnover = 0
    else:
        ix = wt.index | wt_1.index
        tn = wt.reindex(ix) - wt_1.reindex(ix)
        tn[tn.isnull()&tn.index.isin(wt.index)] = wt
        tn[tn.isnull()&tn.index.isin(wt_1.index)] = -wt_1
        turnover = (tn).abs().sum()/2
    turnover_list.append(turnover)
    if (np.sum(turnover_list[-12:])>U):
        print("%0.2f Turnover Constraint Violated: Turnover Limit exceeded"%np.sum(turnover_list[-12:]))
        violated = True
    if (wt<t).any():
        print("Shortsell Constraint Violated: all weights are not greater than parameter t")
        violated = True
    if wt[wt<0].sum()<T:
        print("Max Shortsell Constraint Violated: sum of all weights are not greater than parameter T")
        violated = True
    if wt[wt!=0].count() < np.minimum(P, len(wt)):
        print("Min number of positions Constraint Violated: count of all weights <>0 %i are not greater than parameter P%i"%(wt[wt!=0].count(), np.minimum(P, len(wt))))
        violated = True
    if (wt*Dt).sum()/ (wi*Dt).sum() > delta:
        print("Duration Constraint Violated: wt*Dt/ wi*Dt is greater than parameter delta")
        violated = True
    if (wt*St).sum()/ (wi*St).sum() > chi:
        print("Spread Constraint Violated: wt*St/ wi*St is greater than parameter chi")
        violated = True
    if (wt*(1-Qt)).sum() != 0:
        print("Qualification Constraint Violated: wt*(1-qt) is not zero")
        violated = True
#     if returns - Rlow/ volatility <= np.sqrt(1-eta):
#         print("Max Risk probability Constraint Violated: returns - Rlow/ volatility <= np.sqrt(1-eta)")
            
    return violated
        
        

In [216]:
## Do not change this function, this verifies if final constraints are met
def checkFinalConstraints(Rmin, volmax):
    violated = False
    if np.sum(port_returns)<Rmin:
        print("Total Return Constraint Violated: Total Return is less than Index Return")
        violated = True
    if port_volatility[-1]>volmax:
        print("Volatility Constraint Violated: Vol is higher than Index Vol")
        violated = True
    if sharpe_ratio[-1]<Rmin/volmax:
        print("Sharpe Ratio Constraint Violated: SR is less than Index SR")
        violated = True
        return violated

## Functions to Modify

Now you can fill your logic in functions below. Once you are satisfied, copy the body of the functions below in the template file

**NOTE: When copying functions to template file, only copy the body of the function. DONOT change the function name or signature in template file else submission will not evaluate**

`def getSymbolsToTrade():`  --> This is function name and signature. <br/>
`  return 'G2'`    --> This is function body

#### DONOT change the function name or signature in template file else submission will not evaluate

### Step 1: Fill the asset group you want to model for

In [217]:
#### FILL THE FUNCTIONS BELOW ####

## Step 1: Fill the asset group you want to model for
## This can be 'G1' or 'G2'
def getSymbolsToTrade():
    ################################################
    ####   COPY FROM BELOW INTO TEMPLATE FILE   ####
    ################################################
    
    return 'G1'


### Step 2: Fill the logic to generate weights below

Do not change the inputs to the function. If you want any extra inputs, specify them in `**kwargs`
You can lookup this tutorial on how to use `**kwargs` https://www.geeksforgeeks.org/args-kwargs-python/

In [218]:
## Step 2: Fill the logic to generate weights

## This function takes in the following inputs:
## identifiers: asset identifiers
## reward: reward at time t (based on w(t-1))
## wi: weights to initialize from, if you want to use
## Dt: value of column 'd' per asset
## St: value of column 'S' per asset
## Qt: value of column 'q' per asset
## g: value of constant gamma, read problem descrption for details
## U: value of constant U, read problem descrption for details
## t: value of constant t, read problem descrption for details
## T: value of constant T, read problem descrption for details
## P: value of constant P, read problem descrption for details
## delta: value of constant delta, read problem descrption for details
## chi: value of constant chi, read problem descrption for details
## eta: value of constant eta, read problem descrption for details
## **kwargs: any additional params you want to add can be specified here. kwargs is a dictionary

### do not change the inputs to the function. If you want any extra inputs, specify them in **kwargs
### you can lookup this tutorial on how to use **kwargs https://www.geeksforgeeks.org/args-kwargs-python/

def getWeights(identifiers, reward, wi, Dt, St, Qt, g, U, t, T, P, delta, chi, eta, counter):
    ################################################
    ####   COPY FROM BELOW INTO TEMPLATE FILE   ####
    ################################################
    weights = pd.Series(np.random.random(len(identifiers)), index=identifiers)
    from scipy.optimize import minimize
    index = getSymbolsToTrade()
    if not os.path.isfile('%s.csv'%index):
        downloadFile(index)
    idx_data = pd.read_csv('%s.csv'%index, index_col='TimeStamp')
    idx_data.sort_index(axis=0, level=None, ascending=True, inplace=True)
    idx_data = idx_data.dropna(how='any')
    #idx_data.fillna(0,inplace=True)
    dates = idx_data.index.unique()
    date = dates[counter]
    idx_data = idx_data[idx_data.index == date]
    Qt = idx_data.iloc[:,[6]]['q'].values
    Dt = idx_data.iloc[:,[9]]['d'].values
    wi = idx_data.iloc[:,[18]]['wI'].values
    St = idx_data.iloc[:,[10]]['S'].values
    y_pred = CustomFeatures.newFeature1(idx_data)
    wt = np.zeros(len(idx_data))

    def objective(wt,y_pred):
        result = -1*y_pred.dot(wt)
        return result
    def constrain1(wt):
        return wt.sum() - 1
    def constrain2(wt):
        return (wt - np.maximum(g, 1/float(len(wi)))).any()
    def constrain3(wt):
        return T - wt[wt<0].sum()
    def constrain4(wt):
        return np.minimum(P, len(wt)) - len(wt[wt!=0])
    def constrain5(wt):
        #result = ((Dt*wt).sum(axis=0))
        result = Dt.dot(wt)
        #result2 = (((Dt*wi).sum(axis=1)).sum(axis=0))
        result2 = Dt.dot(wi)
        res = delta - result/result2
        return res
    def constrain6(wt):
        #result = ((wt*St).sum(axis=1)).sum(axis=0)
        result = wt.dot(St)
        #result2 = ((St*wi).sum(axis=1)).sum(axis=0)
        result2 = St.dot(wi)
        res = chi - result/result2
        return res
    def constrain7(wt):
        wtp=wt.astype(np.float32)
        Qtp=Qt.astype(np.float32)
        return (wt*(1 - Qt)).sum() == 0
    from scipy.optimize import Bounds
    bnds = tuple((0.0, 1) for x in wt)
    #bnds = Bounds(bnds)
    con1 = {'type': 'eq', 'fun': constrain1} 
    con2 = {'type': 'eq', 'fun': constrain2} 
    con3 = {'type': 'ineq', 'fun': constrain3} 
    con4 = {'type': 'ineq', 'fun': constrain4} 
    con5 = {'type': 'ineq', 'fun': constrain5} 
    con6 = {'type': 'ineq', 'fun': constrain6} 
    con7 = {'type': 'eq', 'fun': constrain7} 

    cons = [con1,con2,con3,con4,con5,con6,con7]

    solution = minimize(objective,y_pred,wt,method='TNC',\
                        bounds = bnds,constraints=cons)
    weights = weights/weights.sum()
    if (weights>g).any():
        weights[weights>g] = g
        weights = weights/weights.sum()
    return weights

### Step 3: Optional: Fill in the logic to return predictions for return on asset

**Note TRR: Total Return at the end of the period - VERY IMPORTANT!! Don’t use TRR for timestamp t in making predictions for time t. You can use it for time t+1 and later**


In [219]:
## Step 3: Optional: Fill in the logic to return predictions for return on asset
## This function takes in the same inputs as getWeights()

### do not change the inputs to the function. If you want any extra inputs, specify them in **kwargs
### you can lookup this tutorial on how to use **kwargs https://www.geeksforgeeks.org/args-kwargs-python/

def getPrediction(identifiers, wi, Dt, St, Qt, g, U, t, T, P, delta, chi, eta, **kwargs):
    ################################################
    ####   COPY FROM BELOW INTO TEMPLATE FILE   ####
    ################################################
    
    return np.zeros(len(identifiers))

### Step 4: Optional: If your code uses any other helper functions, fill them here

use this in functions above in the following manner: </br>

```CustomFeatures = CustomFeatures()
z = CustomFeatures.newFeature1()```

In [220]:
## Step 4: Optional: If your code uses any other helper functions, fill them here

### again do not change the inputs to the function. If you want any extra inputs, specify them in **kwargs
### you can lookup this tutorial on how to use **kwargs https://www.geeksforgeeks.org/args-kwargs-python/
    
class CustomFeatures():
    def newFeature1(idx_data_test):
    ################################################
    ####   COPY FROM BELOW INTO TEMPLATE FILE   ####
    ################################################
        import matplotlib.pyplot as plt
        index = getSymbolsToTrade()
        if not os.path.isfile('%s.csv'%index):
            downloadFile(index)
        idx_data = pd.read_csv('%s.csv'%index, index_col='TimeStamp')
        idx_data = idx_data.dropna(how='any')
        #idx_data.fillna(0,inplace=True)
        X = idx_data.iloc[:, [5,6,8,9,10,11,14,15,16,17,18,19,21,22,23,24,25,26,27,28,29,30,31,32,33]].values
        y = idx_data.iloc[:, [20]].values
        from sklearn.linear_model import LinearRegression
        Regressor = LinearRegression()
        Regressor.fit(X,y)
        idx_data_test = idx_data_test.iloc[:, [5,6,8,9,10,11,14,15,16,17,18,19,21,22,23,24,25,26,27,28,29,30,31,32,33]].values
        
        from sklearn.preprocessing import StandardScaler
        idx_data_test = pd.DataFrame(idx_data_test)
        idx_data_test = idx_data_test.dropna(how='any')
        #idx_data_test.fillna(0,inplace=True)
        idx_data_test = pd.DataFrame(StandardScaler().fit_transform(idx_data_test), columns=idx_data_test.columns, index=idx_data_test.index)
        y_pred = Regressor.predict(idx_data_test)
        """
        plt.scatter(y_pred, y_test, color = 'blue')
        plt.title('Salary vs Experience (Training set)')
        plt.xlabel('Years of Experience')
        plt.ylabel('Salary')
        plt.show()
        """
        return y_pred

### Step 5: Run the cells below to evaluate. 

The snippet below downloads the correct dataset and lets you explore the dataset.

**Note Remember TRR: Total Return at the end of the period - VERY IMPORTANT!! Don’t use TRR for timestamp t in making predictions for time t. You can use it for time t+1 and later**


In [221]:
## This loads asset data and lets you explore the dataset
## Make sure the worksheet and data are in the same folder

index = getSymbolsToTrade()
if not os.path.isfile('%s.csv'%index):
    downloadFile(index)
idx_data = pd.read_csv('%s.csv'%index, index_col='TimeStamp')
idx_data.sort_index(axis=0, level=None, ascending=True, inplace=True)
print("Data Column Names:")
print(idx_data.columns)
print(index, idx_data.index[0], idx_data.index[-1])
print(idx_data.head(10))

Data Column Names:
Index(['AssetGroup', 'Identifier', 'F2', 'F3', 'F4', 'F5', 'q', 'F7', 'F8',
       'd', 'S', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'wI', 'F19',
       'TRR', 'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28', 'F29',
       'F30', 'F31', 'F32', 'F33'],
      dtype='object')
G1 1998-12-31 2018-11-30
           AssetGroup Identifier        F2                  F3  \
TimeStamp                                                        
1998-12-31         G1   86a1f1ee  bc068363       Capital Goods   
1998-12-31         G1   de5b9bca  e5fb34b9  Telecommunications   
1998-12-31         G1   2a0a4ee3  62835655      Basic Industry   
1998-12-31         G1   a67fb965  81f811c5      Transportation   
1998-12-31         G1   52f697cc  2becce58      Basic Industry   
1998-12-31         G1   bf46c4bb  57847ea5  Telecommunications   
1998-12-31         G1   e39dffda  9c69fc3a  Telecommunications   
1998-12-31         G1   98eed4f4  2becce58      Basic Industry   
1998-1

**The cells below evaluate your functions. DONOT CHANGE ANYTHING BELOW THIS!!**

In [222]:
## DONOT CHANGE ANYTHING BELOW THIS

# empty lists to store returns, volatility and weights of imaginary portfolios
port_returns = []
port_volatility = []
sharpe_ratio = []
asset_weights = []
reward_list = []
turnover_list = []
idx_returns = []

#empty df to store previous period returns
ri = None

In [223]:
### Evaluator to getweights at everytime t and calcuate reward + check if constraints are met

## specifying all the constants
counter = 0
l = 0.1
k = 0.03
g = 0.03
U = 2.5
t= 0
T = 0
P = 80
delta = 0.5
chi = 0.1
eta = 0.95

## initializing arrays
dates = idx_data.index.unique()
# port_returns.append(0)
# port_volatility.append(0)
# sharpe_ratio.append(0)
# asset_weights.append(None)#pd.Series(idx_data[idx_data.index == dates[0]]['wI'].values/100, 
#                            #    index=idx_data[idx_data.index == dates[0]]['Identifier'].values))
# reward_list.append(0)
idx_returns.append(0)
# turnover_list.append(0)
## looping over all dates
while counter < len(dates):
#     import pdb;pdb.set_trace()
    date = dates[counter]
    
    ## load all the data for a date
    date_data = idx_data[idx_data.index == date]
    
    ## get all the identifiers for a date
    cusips = date_data['Identifier'].unique()
    date_data.set_index( date_data['Identifier'], inplace=True)
    #load specific feature info
    wi = pd.Series(date_data['wI']/100, index = date_data['Identifier'])
    Dt = pd.Series(date_data['d'], index = date_data['Identifier'])
    St = pd.Series(date_data['S'], index = date_data['Identifier'])
    qt = pd.Series(date_data['q'], index = date_data['Identifier'])
    ## old weights
    wt_1 = asset_weights[-1] if len(asset_weights)>0 else None    
    ## get new weights

    if wt_1 is None:
        wt = wi.copy()
    else:
        wt = getWeights(cusips, reward_list[-1], wi, Dt, St, qt, g, U, t, T, P, delta, chi, eta, counter) 
    ## calculate reward
    reward = getReward(wt, wt_1, ri, l, k)
    ## store relevant info in their lists
    asset_weights.append(wt)
    reward_list.append(reward)
    
    ## verify if all constraints are met
    
    counter += 1
    
    ## Store end of month returns to calculate reward in next period
    ri = pd.Series(date_data['TRR'], index = date_data['Identifier'])
    ##idx_returns for next period
    idx_returns.append(np.dot(wi, pd.Series(date_data['TRR'], index = wi.index)))


reward = getReward(wt, wt_1, ri, l, k)
print(reward)
## check if contraints on total return and risk are met        
Rmin = np.sum(idx_returns)
volmax = np.std(idx_returns)
if checkFinalConstraints(Rmin, volmax):
    print("ERROR!!!! weights don't meet return/risk limit contraints, exiting")
else:
    print("Portfolio Metrics:")
    print("Total Return: %.2f"%np.sum(port_returns))
    print("Standard Deviation: %.2f"%port_volatility[-1])
    print("Sharpe Ratio: %.2f"%sharpe_ratio[-1])
    

E:\SOFTWARE\lib\site-packages\scipy\optimize\_minimize.py:520: RuntimeWarning: Method TNC cannot handle constraints.
  RuntimeWarning)


-2.4817292545000003 0.0 0.018937349501482275 0 -2.5006666040014824


E:\SOFTWARE\lib\site-packages\scipy\optimize\_minimize.py:520: RuntimeWarning: Method TNC cannot handle constraints.
  RuntimeWarning)


2.613414074794361 0.0 0.019887804285418906 0 2.593526270508942


E:\SOFTWARE\lib\site-packages\scipy\optimize\_minimize.py:520: RuntimeWarning: Method TNC cannot handle constraints.
  RuntimeWarning)


...

2.5256695637529374 0.6585179076148893 0.01896019320874886 21.60543507714878 2.4408575797826995


E:\SOFTWARE\lib\site-packages\scipy\optimize\_minimize.py:520: RuntimeWarning: Method TNC cannot handle constraints.
  RuntimeWarning)


2.2865410112117823 0.6585179076148893 0.020993404382041145 25.077688427848823 2.1996958160682523


E:\SOFTWARE\lib\site-packages\scipy\optimize\_minimize.py:520: RuntimeWarning: Method TNC cannot handle constraints.
  RuntimeWarning)


ValueError: shapes (156,) and (153,) not aligned: 156 (dim 0) != 153 (dim 0)

In [193]:
len(ri)

132

In [194]:
len(wt_1)

129

In [147]:
type(ri)

pandas.core.series.Series

In [148]:
ri = ri[:129]

In [149]:
len(ri)

129

In [165]:
np.dot(wt_1,ri)

ValueError: shapes (129,) and (132,) not aligned: 129 (dim 0) != 132 (dim 0)

In [195]:
length

NameError: name 'length' is not defined